In [1]:
from finiteelementanalysis import visualize as viz
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
import numpy as np
import matplotlib.pyplot as plt

This notebook will show how the solver can fail. Specifically, we apply too large a change in load leading the Newton-Raphson method to fail. 

We will be using D2_nn6_tri with 100 horizontal elements and 2 vertical elements. Our beam is represented by a 30 x 1 rectangle.

In [2]:
ele_type = "D2_nn6_tri" # change this!
nx = 100 # change this to refine the mesh
ny = 1 # change this to refine the mesh
L = 30
H = 1

coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)

Our problem will have a distributed downward load on the top with both left and right ends fixed. Our load and material properties are defined below.

In [3]:
# Identify boundaries
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
# 1. Fix left boundary: both u_x and u_y = 0.
fixed_nodes_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
fixed_nodes_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0.0, 0.0)
fixed_nodes = np.hstack((fixed_nodes_left, fixed_nodes_right))
# Assign distributed load on the right boundary
q = 500
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, -q)
# Assign material properties
E = 1e6
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])
# Assign artificial displacement field
displacement = np.zeros((coords.shape))
for kk in range(0, coords.shape[0]):
    displacement[kk, 0] = coords[kk, 0] * 0.01

# Analytical solution
def solution(x, load_factor):
    w = q*load_factor
    I = H**3/12
    y = 0.5 - w * x**2 / (24*E*I) * (L-x)**2
    return y

# Animated plot
fname = "hw_solver_large_deformation.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)

If we try and apply the load all at once, we run into an issue where the solver does not converge.

In [4]:
# run the example to look at the results

nr_num_steps = 1
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

fname = "large-def.gif"
viz.make_deformation_gif(solution, displacements_all, coords, connect, ele_type, fname)

Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=9.261322e-01, tolerance=1.000000e-09
Iteration 2, Correction=1.424711e+00, Residual=6.517858e+03, tolerance=1.000000e-09


/Users/ryanengle/Documents/College/Graduate School/Classes/ME700 Lejeune/My Projects/me700-hw3/src/finiteelementanalysis/solver.py:133: MatrixRankWarning: Matrix is exactly singular
  d_displacement = spla.spsolve(K_sparse, R)


Iteration 3, Correction=0.000000e+00, Residual=nan, tolerance=1.000000e-09


ValueError: Axis limits cannot be NaN or Inf